In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install git+https://github.com/huggingface/transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate

#from 20.5.2024
# !pip install audiomentations[extras]
# !pip install mutagen
# !pip install srt

In [3]:
from huggingface_hub import login
# load the token from txt file
with open("./tokens/HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/user_7542/.cache/huggingface/token
Login successful


In [4]:
import os
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json
#from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, RoomSimulator
import srt
import re
from IPython.display import clear_output

#our libraries
from global_variables.training_vars import *
from global_variables.folders import *
from parashat_hashavua_dataset import parashat_hashavua_dataset
from nikud_and_teamim import just_teamim,remove_nikud


In [5]:
processor = WhisperProcessor.from_pretrained(BASE_MODEL_NAME, language="hebrew", task="transcribe")

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# # Check how many tokens needed for each nikud/taam
# chars = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
# for c in chars:
#     print(f"{c}: {len(processor.tokenizer.encode(c))-5}")


In [7]:
tokens_added = (len(processor.tokenizer.encode('֟'))==6) # check if the tokens were already added
if ADDTOKENS and not tokens_added: # add the tokens if they weren't already added
    
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim
    
    processor.tokenizer.add_tokens(new_tokens)

In [8]:
if NEWDATA:
    train_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)

else:
    train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)

Loading ashkenazi nusach (1/4): 100%|██████████| 500/500 [00:02<00:00, 230.20it/s]


Num of missing files in ashkenazi nusach:  2


Loading maroko nusach (2/4): 100%|██████████| 500/500 [00:01<00:00, 368.66it/s]


Num of missing files in maroko nusach:  5


Loading yerushalmi nusach (3/4): 100%|██████████| 500/500 [00:01<00:00, 316.22it/s]


Num of missing files in yerushalmi nusach:  5


Loading bavly nusach (4/4): 100%|██████████| 500/500 [00:02<00:00, 211.89it/s]

Num of missing files in bavly nusach:  6


In [9]:
# check the data of Pocket Torah
# train_data.check_failed_files_of_old_data()

In [10]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [11]:
if NEWDATA:
    val_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)
else:
    val_data = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM, processor=processor)
    

print(len(train_data))
print(len(val_data))

Loading ashkenazi nusach (1/4): 100%|██████████| 500/500 [00:01<00:00, 290.29it/s]


Num of missing files in ashkenazi nusach:  3


Loading maroko nusach (2/4): 100%|██████████| 500/500 [00:01<00:00, 331.37it/s]


Num of missing files in maroko nusach:  10


Loading yerushalmi nusach (3/4): 100%|██████████| 500/500 [00:01<00:00, 300.24it/s]


Num of missing files in yerushalmi nusach:  7


Loading bavly nusach (4/4): 100%|██████████| 500/500 [00:01<00:00, 313.48it/s]

Num of missing files in bavly nusach:  26
100000
488


In [12]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [14]:
import concurrent.futures
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

2024-06-06 13:53:08.363929: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 13:53:08.386466: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he"




# # initialize the last layer of the model:
# model.proj_out.__init__(model.proj_out.in_features, len(processor.tokenizer))

# # add dropout
if DROPOUT:
    model.config.attention_dropout = DROPOUT
    model.config.dropout = DROPOUT
    model.config.activation_dropout = DROPOUT


if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

In [16]:
# MODEL_NAME

In [17]:
from transformers import Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS, 
    eval_steps=EVAL_STEPS,   
    logging_steps=25, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    run_name=RUN_NAME, # It doesn't work
)


/home/user_7542/.local/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from transformers import Seq2SeqTrainer, TrainerCallback


class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [EvaluateFirstStepCallback()] if EVALUATE_FIRST_STEP else None
)



comet_ml is installed but `COMET_API_KEY` is not set.
max_steps is given, it will override any value given in num_train_epochs


In [19]:
processor.save_pretrained(training_args.output_dir)

[]

In [20]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [21]:
flags_warnings()

trainer_state = trainer.train()

!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only
!!!TEST-MODE!!! 	 	 to test the code only


Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
1,No log,5.817735,115.625628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
100,0.173200,0.871102,80.196827,0.288276,0.307332,0.296318,0.327251,0.350221,0.337000,0.343237,0.366576,0.353157,0.464669,0.497407,0.478551,0.272727,0.300000,0.285714,0.933333,0.933333,0.933333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


best f1 for Exact: 0
best pred: גם כל חולי וכל מכה, והנה מן היור כאשר והייתה לכהן כמנחה.
best label: ג֤ם כל־חלי֙ וכל־מכ֔ה והנ֣ה מן־היא֗ר כֽאש֛ר והֽית֥ה לכה֖ן כמנחֽה׃

worst f1 for Exact: 0
worst pred: גם כל חולי וכל מכה, והנה מן היור כאשר והייתה לכהן כמנחה.
worst label: ג֤ם כל־חלי֙ וכל־מכ֔ה והנ֣ה מן־היא֗ר כֽאש֛ר והֽית֥ה לכה֖ן כמנחֽה׃

worst f1 for Letter_Shift: 0
worst pred: גם כל חולי וכל מכה, והנה מן היור כאשר והייתה לכהן כמנחה.
worst label: ג֤ם כל־חלי֙ וכל־מכ֔ה והנ֣ה מן־היא֗ר כֽאש֛ר והֽית֥ה לכה֖ן כמנחֽה׃

worst f1 for Word_Level: 0
worst pred: גם כל חולי וכל מכה, והנה מן היור כאשר והייתה לכהן כמנחה.
worst label: ג֤ם כל־חלי֙ וכל־מכ֔ה והנ֣ה מן־היא֗ר כֽאש֛ר והֽית֥ה לכה֖ן כמנחֽה׃

worst f1 for Word_Shift: 0
worst pred: גם כל חולי וכל מכה, והנה מן היור כאשר והייתה לכהן כמנחה.
worst label: ג֤ם כל־חלי֙ וכל־מכ֔ה והנ֣ה מן־היא֗ר כֽאש֛ר והֽית֥ה לכה֖ן כמנחֽה׃

Time taken for each part:
Decode calculation: 18.045287132263184 seconds
WER calculation: 0.01890254020690918 seconds
best f1 for Exact:

KeyboardInterrupt: 

In [ ]:
kwargs = {
    # "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he-cantillation",
    "finetuned_from": BASE_MODEL_NAME,
    "tasks": "automatic-speech-recognition-cantillation",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.save_model()

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


In [ ]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/cantillation/Teamim-medium_Random-True_OriginalData/commit/bc4ad6d5f66e0fc830c7d19a6ba21a4eb62fa73b', commit_message='End of training', commit_description='', oid='bc4ad6d5f66e0fc830c7d19a6ba21a4eb62fa73b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.lr_scheduler.get_lr()

[4.081632653061225e-09, 4.081632653061225e-09]

In [ ]:
processor.tokenizer.decode(train_data[26]["labels"])

'<|startoftranscript|><|he|><|transcribe|><|notimestamps|>לחטֽאת׃ ולז֣בח השלמים֮ שש֥ה עש֖ר אדנ֑ים א֖יש אשר־ נשא֣ו מש֔ה ב֖א בק֗ש להדֽיחך֙ ע֥בד לֽמו׃<|endoftext|>'

In [ ]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="./markdown_files/training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_avg_f1_Exact']
recall = history['eval_avg_recall_Exact']
precision = history['eval_avg_precision_Exact']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="./markdown_files/training_log_new.md")


In [ ]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('./markdown_files/training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|
| 03/21/2024, 08:40:08 | ./Teamim-AllNusah-whisper-small_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.021550073864677687 | 8.0 | 100000 | 0.184482142329216 | 0.8996304230096901 | 0.8996085856138029 | 0.9003176340674156 |
| 03/21/2024, 13:03:45 | ./Teamim-AllNusah-whisper-base_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.03545441636119969 | 8.0 | 100000 | 0.20118673145771027 | 0.8617680742203201 | 0.8626021186921445 | 0.8617811978432426 |
| 03/21/2024, 16:41:01 | ./Teamim-AllNusah-whisper-tiny_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.04949023559953086 | 8.0 | 100000 | 0.21391823887825012 | 0.8324883831395741 | 0.8338224092406237 | 0.8322131178574591 |
| 03/23/2024, 07:00:56 | ./Teamim-AllNusah-whisper-medium_Random-True_Mid | 8 | 1 | 1e-05 | 1000 | 100000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 10000 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.016411178880098305 | 8.0 | 100000 | 0.19175758957862854 | 0.9241458540823655 | 0.9248030309610401 | 0.9241210924324863 |
| 05/15/2024, 04:43:26 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 20000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 10000 | 200 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.06837314007226378 | 1.6 | 20000 | 0.32565802335739136 | 0.7752270688239551 | 0.7795095778021163 | 0.7720285537597026 |
| 05/16/2024, 20:40:49 | ./Teamim-medium_Random-True_OriginalData | 8 | 1 | 1e-05 | 100 | 5000 | True | {'use_reentrant': False} | True | steps | 32 | True | 225 | 1000 | 500 | 25 | ['tensorboard'] | True | avg_f1_Exact | True | True | 0.1853820906639099 | 0.4 | 5000 | 0.14098824560642242 | 0.892768311387271 | 0.8960657159022651 | 0.890549476438386 |


| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




In [ ]:
# run a web server to see the tensorboard
!tensorboard --logdir ./whisper-medium-he-teamim-aviv-base --port 6006 --bind_all

2024-05-16 20:40:50.477979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 20:40:51.051523: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-16 20:40:51.805602: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-16 20:40:51.834281: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Fo

# Test the model

In [ ]:
# load the model
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

# load the test data
test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)

# create the data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# evaluate the model
results = trainer.evaluate() # we use evaluate to get the metrics
print(results)
# save the results to a json file
# create the results file
with open(f"results_{MODEL_NAME.split('/')[-1]}.json", 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
{'eval_loss': 0.7434155344963074, 'eval_wer': 12.154756685664182, 'eval_avg_precision_Exact': 0.9045080330977024, 'eval_avg_recall_Exact': 0.9046995984902677, 'eval_avg_f1_Exact': 0.9041808511811215, 'eval_avg_precision_Letter_Shift': 0.9262739837538204, 'eval_avg_recall_Letter_Shift': 0.9265417247288052, 'eval_avg_f1_Letter_Shift': 0.9259759561670436, 'eval_avg_precision_Word_Level': 0.9287635843313802, 'eval_avg_recall_Word_Level': 0.9291329677264785, 'eval_avg_f1_Word_Level': 0.928526987815817, 'eval_avg_precision_Word_Shift': 0.9744189603225837, 'eval_avg_recall_Word_Shift': 0.9756642194342305, 'eval_avg_f1_Word_Shift': 0.9746023104771994, 'eval_precision_median_exact': 1.0, 'eval_recall_median_exact': 1.0, 'eval_f1_median_exact': 1.0, 'eval_precision_max_exact': 1.0, 'eval_recall_max_exact': 1.0, 'eval_f1_max_exact': 1.0, 'eval_precision_min_Exact': 0.0, 'eval_recall_min_Exact': 0.0, 'eval_f1_min_Exact': 0.0, 'eval_precision_min_Letter_Shift': 0.0, 'eval_recall_min_Letter_Shift': 0.0, 'eval_f1_min_Letter_Shift': 0.0, 'eval_precision_min_Word_Level': 0.0, 'eval_recall_min_Word_Level': 0.0, 'eval_f1_min_Word_Level': 0.0, 'eval_precision_min_Word_Shift': 0.0, 'eval_recall_min_Word_Shift': 0.0, 'eval_f1_min_Word_Shift': 0.0, 'eval_runtime': 1534.2005, 'eval_samples_per_second': 1.755, 'eval_steps_per_second': 0.055}